In [1]:
import requests
import pandas as pd
import numpy as np
import time

In [2]:
# 매번 사용한 api_key값과 header를 생성한다.
import requests
 
api_key = "RGAPI-7d4b52a9-5f5a-4b61-a0b8-f98484f88e77"
header = {"X-Riot-Token" : api_key}

In [3]:
# 첼린저,그랜드마스터,마스터의 솔로랭크 정보를 가져오기 위한 함수 생성

def league_v4_tier(tier):
    # 솔랭 기준
    queue = 'RANKED_SOLO_5x5'
 
    if tier == 'challenger' :
        url = f"https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{queue}"
    elif tier == 'grandmaster':
        url = f"https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{queue}"
    else:
        url = f"https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{queue}"
        
    return requests.get(url, headers=header)

In [ ]:
challenger_data = pd.DataFrame(league_v4_tier('challenger').json())

In [ ]:
# 첼린저 티어의 솔로랭크 정보중 entries에 들어 있는 summonerId 및 승패정보를 테이블로 담기

challenger_df = pd.DataFrame([challenger_data['entries'][0]])
data_size = len(challenger_data)

for i in range(1,data_size):
    append_df = pd.DataFrame([challenger_data['entries'][i]])
    challenger_df = pd.concat([challenger_df,append_df],ignore_index = True)

challenger_df['tier'] = 'CHALLENGER'
challenger_df

In [4]:
# summonerid로 소환사 정보인 puuid를 가져오기 위한 API코드
def summoners(encryptedSummonerId):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{encryptedSummonerId}"
    
    return requests.get(url, headers = header)

In [ ]:
# challenger_df에 추가적으로 컬럼을 생성하고 temp값을 넣어 둔다.(주석처리해둔다)

challenger_df['accountId'] = 'temp_1'
challenger_df['puuid'] = 'temp_2'
challenger_df['profileIconId'] = 'temp_3'
challenger_df['revisionDate'] = 'temp_4'
challenger_df['summonerLevel'] = 'temp_5'

In [ ]:
# 유저마다 puuid,accountid등을 가져와서 top_class_df에 넣는다.(첼린저만 가져오는 코드)
# 해당코드는 한번 생성한 뒤에는 실행되지 않게끔 주석처리해둔다.

for i in range(len(challenger_df)):
    encryptedSummonerId = challenger_df['summonerId'].loc[i]
    summoners_info = summoners(encryptedSummonerId).json()

    acid = summoners_info['accountId']
    puid = summoners_info['puuid']
    proid = summoners_info['profileIconId']
    re_date = summoners_info['revisionDate']
    summo_level = summoners_info['summonerLevel']

    challenger_df.loc[i,'accountId'] = acid
    challenger_df.loc[i,'puuid'] = puid
    challenger_df.loc[i,'profileIconId'] = proid
    challenger_df.loc[i,'revisionDate'] = re_date
    challenger_df.loc[i,'summonerLevel'] = summo_level
    
    print(i,summoners_info['id'])
    if i%100 == 0:
        challenger_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_summoner_challenger_tb.csv")
        print("중간저장 합니다.")
    else:
        pass

    time.sleep(1)

challenger_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_summoner_challenger_tb.csv")
challenger_df

In [5]:
# challenger데 tb를 불러온다
challenger_user_basic_df = pd.read_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_summoner_challenger_tb.csv")
challenger_user_basic_df

,Unnamed: 0,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,tier,accountId,puuid,profileIconId,revisionDate,summonerLevel
0,0,rzkFD309zDeQsv_7J62emy_cssSKOGG8y89scRY9O3E_pBbk,1197,I,305,261,True,False,False,False,CHALLENGER,FGBFPlbHedFgFM-EBmaH_VU8BKhsToGFq4fIKjv7hSnTbx...,FCXSAO0fTl2hbxK8whdiN0P-a2Ub_7pzAjA5uWVuajKm2y...,3,1722627139722,118
1,1,HkiD1-r1Ag34BL081D4YTNTK_PNCWpZVjG-GIRo9e1Z1lz0,1208,I,218,175,False,False,False,True,CHALLENGER,2uDv38Lq4-kbXtVuYJNHxOoX_iXR_IAU-MlaD69nwiVWYas,9BdGvJe5JDLTVCmSDpvojDdj8wFvJIQBFIcXPalFUT3iTb...,5894,1722618647136,809
2,2,DDqi6mcntdttdhZJLoVCUeKpYMoXNbUEkaY3rbX4mXD6hl...,1180,I,150,100,False,False,False,True,CHALLENGER,53leYzcUhs0Eg5rRN20wf3xzKy2-y3ZNKyoCC446UaxbNG...,zkdc2WnbAj_Z0g7mUJraq84MoD67CChrvcjaXGFIsvEVde...,7,1722324106000,206
3,3,zt69h22Oscroc-5QTaueAsVdL6nhUmFbq1-AyO4d1Rs-lC...,1103,I,119,73,False,False,False,False,CHALLENGER,kThaEym9hV6VT-39-m-LYsm5S5iqAR_ZmY3qwqCVQOQZzL...,GQhSr8VFpJfD4rkUjRRy_TTClCh6Ir2EZZip_ufsvrQA8y...,3018,1722549911000,120
4,4,BLbdCfQG5fb1D0HZyNfNIMROthIrPO4LjNqVpEqU_cbeVb...,1232,I,273,227,True,False,False,False,CHALLENGER,KRs6-ZjLxLg-Sx5FbVm_CB-YWBDObjSGT7cnF7QGh7Y_mp...,QP4WVEBrkYW9bTX_gwp9i6xxEF0RO7GY39TnPGr0DHfZpM...,6668,1722612246120,611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,jwyUnVUhiSLk_dw_s-T_jSZKpHxACz-uhBZWa4Zpv64KGos,1242,I,233,181,True,False,False,False,CHALLENGER,gYJb4i-8eTEViTJpOaKWgJrh1jQ3z5HiCMNs1nNXfol2kL8,J9hZECWsVc-uKOkY6DCByvSZdomF_-mAu96xfvpk3qVMfR...,6362,1722594622062,1001
296,296,UtandpFBeupewx9d2-R0L8FhUklOzocEIIuXRAYDQvFyg74,1148,I,138,91,False,False,False,False,CHALLENGER,PWQ8eP5d0OCZ7zwD-eFAGSLerjMdN1cLQOfwUsqmocPbn4c,E5k3CFhr7drCO6HbJpHgeC4WeiuW4nITWLH5CNhg_vjCWC...,588,1722627899350,561
297,297,iji0Cwskw6VYepj88fupN4w91y1oYwp32vfgqmPAKJ--oG4,1134,I,268,223,False,False,True,False,CHALLENGER,hYMZpxOkhs1xHait5bqssuZFcN7GVUXwIFkx-Mm9blYwbI...,J5n1g8us8IYL10VU4V1LY6nnAKo1xSAfYw3M5hOFMyh09O...,5871,1722621787492,745
298,298,5wrQxegh7eoI9Xd0gQteyGxvQpo8PcznbBw-WkRcwZxkaaI,1626,I,187,124,True,False,False,False,CHALLENGER,KIkC0cU1UcGWZwMpLuCMKr8QErT956MKPw5XNnCpQQMHQOk,NEj4VLg04NJ7erJQfYokC1LKA2Q4fu2KXPDZjx2DUtj557...,1456,1722448950000,398


In [6]:
# puuid를 활용하여 match list를 불러오는 함수

def match_list(puuid):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start={0}&count={100}"
    
    return requests.get(url, headers=header)

In [ ]:
# puuid를 사용하여 유저별로 100개의 match list를 불러오고 중복된 리스트는 제거하여 14,773명의 전체 경기 리스트를 생성(챌린저유저만,주석처리)

challenger_match_list = []

for i in range(len(challenger_user_basic_df['puuid'])):
    puuid = challenger_user_basic_df['puuid'].loc[i]
    add_match_list = match_list(puuid).json()
    challenger_match_list = list(set(challenger_match_list + add_match_list))
    print(i,puuid,len(challenger_match_list))
    time.sleep(1)

challenger_match_list

In [ ]:
# puuid로 생성한 challenger match list를 테이블형태로 변환하여 CSV파일로 저장(주석처리)

match_df = pd.DataFrame(challenger_match_list)
match_df.rename(columns={0 : 'user_match_list'},inplace=True)
match_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_match_tb.csv")
match_df

In [8]:
# 저장된 challenger user의 match list 불러오기

challenger_user_match_df = pd.read_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_match_tb.csv")
challenger_user_match_df = challenger_user_match_df.drop(['Unnamed: 0'],axis=1)
challenger_user_match_df

,user_match_list
0,KR_7125420162
1,KR_7152534895
2,KR_7149541209
3,KR_7140866765
4,KR_7188754497
...,...
14278,KR_7198549175
14279,KR_7132037738
14280,KR_7129773474
14281,KR_7124872144


In [9]:
def match(matchId):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}"
    
    return requests.get(url, headers=header)

In [ ]:
match_data_metadata_df = pd.DataFrame()
match_data_participants_df = pd.DataFrame()
match_data_teams_df = pd.DataFrame()
match_data_info_df = pd.DataFrame()

for i in range(32400):
    # matchid를 통해 match정보를 가져오기
    matchId = user_match_df['user_match_list'].loc[i+20456]
    respones_result = match(matchId)
    match_data_respones = respones_result.status_code
    match_data_row = respones_result.json()

    if match_data_respones == 200:
        # match정보중 metadata를 전처리하여 match별로 쌓아서 테이블로 생성--------------------------------------
        for num in range(len(match_data_row['metadata']['participants'])):
            player_number = 'player_' + str(num+1)
            match_data_row['metadata'][str(player_number)] = match_data_row['metadata']['participants'][num]
        del(match_data_row['metadata']['participants'])
        
        match_data_metadata = pd.DataFrame(match_data_row['metadata'],index=[0])
        match_data_metadata_df = pd.concat([match_data_metadata_df,match_data_metadata],ignore_index = True)
        

        # match정보중 participants를 전처리하여 match별로 쌓아서 테이블로 생성----------------------------------------------
        match_data_participants = pd.DataFrame(match_data_row['info']['participants'])
        match_data_participants['matchId'] = matchId
        match_data_participants_df = pd.concat([match_data_participants_df,match_data_participants],ignore_index = True)
        

        # match정보중 teams를 전처리하여 match별로 쌓아서 테이블로 생성--------------------------------
        match_data_teams = pd.DataFrame(match_data_row['info']['teams'])
        match_data_teams['matchId'] = matchId
        match_data_teams_df = pd.concat([match_data_teams_df,match_data_teams],ignore_index = True)


        # match정보중 info를 전처리하여 match별로 쌓아서 테이블로 생성------------------------------
        del match_data_row['info']['participants']
        del match_data_row['info']['teams']
        match_data_info = pd.DataFrame(match_data_row['info'],index=[0])
        match_data_info['matchId'] = matchId
        match_data_info_df = pd.concat([match_data_info_df,match_data_info],ignore_index = True)
        
        print(i,matchId)
        if i%100 == 0:
            match_data_metadata_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_metadata_tb_2.csv")
            match_data_participants_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_participants_tb_2.csv")
            match_data_teams_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_teams_tb_2.csv")
            match_data_info_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_info_tb_2.csv")
            print("중간저장 합니다.")
        else:
            pass
    else:
        pass

    time.sleep(1)